Mounting Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Importing neccasary Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn import metrics
import numpy as np
import itertools
import matplotlib.pyplot as plt

Importing CSV file into a pandas Dataframe

In [ ]:
check = pd.read_csv('/content/drive/MyDrive/Capstone_1/processed_input.csv')

Checking Shape of data

# This is five model Approach. We decided to apply a strategy of group and concuqer

# Approach


1.   The entire data was vectorized using TF/IDF method 
2.   The corpus was passed through a list of ML models
3.   We now know that the data is highly imbalanced and Group 0 and Group8
     have the highest occurances
4.   The model which gave the highest Accuracy for Group 0 and Group 8 was   selected in the intial run.
5.   In our case we selected Logistic regression. After this we again re-ran the selected model. Sorted the group basis there F1 score and then didvided the entire data in 4 groups. Each group would have four member and left out group would be marked as others
6. We decided to build 5 models and the data will through each model in a water fall manner.
7. Groups having count of less than 2 were dropped as we can consider it as an exception






# Loading Data

In [ ]:
check = pd.read_csv('/content/drive/MyDrive/Capstone_1/processed_input.csv')
pd.set_option("display.max_rows", None, "display.max_columns", None,'display.max_colwidth', None)

# Dropping data which could not be translated

In [ ]:
check = check[(check['Action_Flag'] != 'Drop')]

# Checking counts of Group 0 and rest and resetting index

In [ ]:
check['isGRP_0'].value_counts(dropna=False)
check.reset_index(drop=True,inplace=True)

In [ ]:
# Creating a copy

In [ ]:
check_x = check.copy()

# Renaming Assignment group as Assig_group

In [ ]:
check_x.rename(columns= {'Assignment group':'Assig_group'} , inplace =True)

# Removing groups having 1,2 and 3 counts

In [ ]:
remove_list =['GRP_73','GRP_35','GRP_72','GRP_70','GRP_64','GRP_67','GRP_61','GRP_32','GRP_71','GRP_57','GRP_54','GRP_69']
remove_filt = (check_x['Assig_group'].isin(remove_list))

In [ ]:
check_x= check_x.loc[~remove_filt]

In [ ]:
# check_x['Assig_group'].value_counts(dropna = False)

In [ ]:
check_x.reset_index(drop=True,inplace=True)

# Dowloading/Importing Libraries

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Converting the data into List. For further actions

In [ ]:
corpus_x = []
for i in range(0, len(check_x)):
    review = check_x['p_long_desc'][i]
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_x.append(review)

In [ ]:
len(corpus_x)

8164

# Converting to TF/IDF vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(ngram_range=(1,3),min_df=2)
X=tfidf_v.fit_transform(corpus_x).toarray()

In [ ]:
# Label encoder 
y=check_x['Assig_group']
y.value_counts(dropna=False)

GRP_0     3785
GRP_8      661
GRP_24     289
GRP_9      251
GRP_2      238
GRP_12     232
GRP_19     211
GRP_3      200
GRP_6      183
GRP_13     140
GRP_10     134
GRP_5      128
GRP_25     116
GRP_14     115
GRP_33     107
GRP_4       99
GRP_29      94
GRP_18      87
GRP_16      80
GRP_17      78
GRP_7       68
GRP_31      62
GRP_34      61
GRP_26      54
GRP_40      45
GRP_28      44
GRP_41      40
GRP_15      38
GRP_42      37
GRP_20      35
GRP_45      34
GRP_1       31
GRP_22      31
GRP_11      29
GRP_21      28
GRP_47      27
GRP_62      25
GRP_23      23
GRP_60      20
GRP_39      19
GRP_27      18
GRP_30      18
GRP_37      16
GRP_36      15
GRP_44      15
GRP_50      13
GRP_65      11
GRP_53      11
GRP_51       8
GRP_55       8
GRP_52       8
GRP_48       7
GRP_46       5
GRP_43       5
GRP_59       5
GRP_66       4
GRP_56       3
GRP_63       3
GRP_68       3
GRP_49       3
GRP_38       3
GRP_58       3
Name: Assig_group, dtype: int64

# Train test split of 70:30

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

# Testing data on MultinomialNB,LogisticRegression,DecisionTreeClassifier,RandomForestClassifier,ExtraTreesClassifier using One vs One approach

0 - MultinomialNB
1 - LogisticRegression
2 - DecisionTreeClassifier
3 - RandomForestClassifier
4 - ExtraTreesClassifier

Extra tree classifier had the highest accuracy of 65%

In [ ]:
%%time
import gc 
from IPython.display import display, HTML, Markdown, clear_output
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ml_score_board = pd.DataFrame(columns=['Model'])
ml_model_dict = {}

#ml_pipelines = [Pipeline([('estimator',LogisticRegression(solver='lbfgs',random_state=0, multi_class='ovr'))])]
#ml_pipelines = [Pipeline([('estimator',LogisticRegression(multi_class='multinomial', solver='newton-cg',random_state=0))])
ml_pipelines = [Pipeline([('estimator', OneVsOneClassifier(MultinomialNB()))]),
                Pipeline([('estimator',LogisticRegression(multi_class='multinomial', solver='newton-cg',random_state=0))]),
                Pipeline([('estimator',OneVsOneClassifier(DecisionTreeClassifier()))]),
                Pipeline([('estimator',OneVsOneClassifier(RandomForestClassifier()))]),
                Pipeline([('estimator',OneVsOneClassifier(ExtraTreesClassifier()))])]
                #Pipeline([('estimator',OneVsOneClassifier(KNeighborsClassifier()))])]
for ml_model in ml_pipelines:
  #estimator_name = ml_model['estimator'].__class__.__name__
  estimator_name = ml_model['estimator'].__class__.__name__

  if estimator_name == 'SVC':
    estimator_name = 'SVC-' + ml_model['estimator'].kernel
  print(f"\nTraining {estimator_name}...")

  ml_model.fit(X_train,y_train)
  #ml_model.fit(X_train_res,y_train_res)
  pred_build = ml_model.predict(X_train)
  #pred_build = ml_model.predict(X_train_res)
  score_build = metrics.accuracy_score(y_train, pred_build)
  #score_build = metrics.accuracy_score(y_train_res, pred_build)
  train_M = metrics.confusion_matrix(y_train, pred_build)
  #train_M = metrics.confusion_matrix(y_train_res, pred_build)
  pred = ml_model.predict(X_test)
  score = metrics.accuracy_score(y_test, pred)
  test_M = metrics.confusion_matrix(y_test, pred)
  clear_output(wait=True)
  ml_score_board = ml_score_board.append({'Model' : estimator_name, 'Build Acc' : score_build, 'Val Acc' : score},ignore_index=True)
  ml_model_dict[estimator_name] = ml_model
  gc.collect()
  display(ml_score_board)
  #print(train_M)
  #print(test_M)

,Model,Build Acc,Val Acc
0,OneVsOneClassifier,0.568634,0.560896
1,LogisticRegression,0.648970,0.600407
2,OneVsOneClassifier,0.927873,0.617108
3,OneVsOneClassifier,0.927174,0.626069
4,OneVsOneClassifier,0.927873,0.635438


CPU times: user 3h 1min 10s, sys: 20min 3s, total: 3h 21min 13s
Wall time: 2h 32min 12s


# Re-Running Extra Tree classifier to test the accuracy/F1 score on Group 0. It is pre decided the model giving the highest accuracy on group 0 will be selected irespective of over all accuracy.
# ET has accuracy 0f 64%

In [ ]:
%%time
import gc 
from IPython.display import display, HTML, Markdown, clear_output
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ml_score_board = pd.DataFrame(columns=['Model'])
ml_model_dict = {}

#ml_pipelines = [Pipeline([('estimator',LogisticRegression(solver='lbfgs',random_state=0, multi_class='ovr'))])]
#ml_pipelines = [Pipeline([('estimator',LogisticRegression(multi_class='multinomial', solver='newton-cg',random_state=0))])
ml_pipelines = [Pipeline([('estimator',OneVsOneClassifier(ExtraTreesClassifier()))])]
for ml_model in ml_pipelines:
  #estimator_name = ml_model['estimator'].__class__.__name__
  estimator_name = ml_model['estimator'].__class__.__name__

  if estimator_name == 'SVC':
    estimator_name = 'SVC-' + ml_model['estimator'].kernel
  print(f"\nTraining {estimator_name}...")

  ml_model.fit(X_train,y_train)
  #ml_model.fit(X_train_res,y_train_res)
  pred_build = ml_model.predict(X_train)
  #pred_build = ml_model.predict(X_train_res)
  score_build = metrics.accuracy_score(y_train, pred_build)
  #score_build = metrics.accuracy_score(y_train_res, pred_build)
  train_M = metrics.confusion_matrix(y_train, pred_build)
  #train_M = metrics.confusion_matrix(y_train_res, pred_build)
  pred = ml_model.predict(X_test)
  score = metrics.accuracy_score(y_test, pred)
  test_M = metrics.confusion_matrix(y_test, pred)
  clear_output(wait=True)
  ml_score_board = ml_score_board.append({'Model' : estimator_name, 'Build Acc' : score_build, 'Val Acc' : score},ignore_index=True)
  ml_model_dict[estimator_name] = ml_model
  gc.collect()
  display(ml_score_board)
  #print(train_M)
  #print(test_M)

,Model,Build Acc,Val Acc
0,OneVsOneClassifier,0.927546,0.648163


CPU times: user 1h 16min 39s, sys: 8.57 s, total: 1h 16min 48s
Wall time: 1h 16min 30s


# GROUP 0 has an f1 score of 78% 

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

       GRP_0       0.65      0.96      0.78      1158
       GRP_1       0.25      0.11      0.15         9
      GRP_10       0.67      0.05      0.09        40
      GRP_11       0.00      0.00      0.00        10
      GRP_12       0.45      0.38      0.41        63
      GRP_13       0.65      0.35      0.45        49
      GRP_14       0.76      0.33      0.46        40
      GRP_15       0.00      0.00      0.00        16
      GRP_16       1.00      0.04      0.07        27
      GRP_17       0.96      0.92      0.94        26
      GRP_18       0.75      0.24      0.36        25
      GRP_19       0.34      0.16      0.22        61
       GRP_2       0.69      0.40      0.51        60
      GRP_20       0.00      0.00      0.00        11
      GRP_21       1.00      0.11      0.20         9
      GRP_22       0.00      0.00      0.00         8
      GRP_23       0.50      0.22      0.31         9
      GRP_24       0.80    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Checking on Logistic Regression

In [ ]:
%%time
import gc 
from IPython.display import display, HTML, Markdown, clear_output
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ml_score_board = pd.DataFrame(columns=['Model'])
ml_model_dict = {}

#ml_pipelines = [Pipeline([('estimator',LogisticRegression(solver='lbfgs',random_state=0, multi_class='ovr'))])]
ml_pipelines = [Pipeline([('estimator',LogisticRegression(multi_class='multinomial', solver='newton-cg',random_state=0))])]
#ml_pipelines = [Pipeline([('estimator',OneVsOneClassifier(ExtraTreesClassifier()))])]
for ml_model in ml_pipelines:
  #estimator_name = ml_model['estimator'].__class__.__name__
  estimator_name = ml_model['estimator'].__class__.__name__

  if estimator_name == 'SVC':
    estimator_name = 'SVC-' + ml_model['estimator'].kernel
  print(f"\nTraining {estimator_name}...")

  ml_model.fit(X_train,y_train)
  #ml_model.fit(X_train_res,y_train_res)
  pred_build = ml_model.predict(X_train)
  #pred_build = ml_model.predict(X_train_res)
  score_build = metrics.accuracy_score(y_train, pred_build)
  #score_build = metrics.accuracy_score(y_train_res, pred_build)
  train_M = metrics.confusion_matrix(y_train, pred_build)
  #train_M = metrics.confusion_matrix(y_train_res, pred_build)
  pred = ml_model.predict(X_test)
  score = metrics.accuracy_score(y_test, pred)
  test_M = metrics.confusion_matrix(y_test, pred)
  clear_output(wait=True)
  ml_score_board = ml_score_board.append({'Model' : estimator_name, 'Build Acc' : score_build, 'Val Acc' : score},ignore_index=True)
  ml_model_dict[estimator_name] = ml_model
  gc.collect()
  display(ml_score_board)
  #print(train_M)
  #print(test_M)

,Model,Build Acc,Val Acc
0,LogisticRegression,0.646482,0.614286


CPU times: user 6min 56s, sys: 1min 32s, total: 8min 29s
Wall time: 2min 10s


# Logistic regression gave us an F1 score of 0.75 while ET gave 0.79. On further evaluation we found that Logistic regression was better than Extra Trees

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

       GRP_0       0.61      0.99      0.75      1159
       GRP_1       0.00      0.00      0.00         9
      GRP_10       0.00      0.00      0.00        37
      GRP_11       0.00      0.00      0.00        11
      GRP_12       0.57      0.25      0.35        63
      GRP_13       0.62      0.25      0.35        53
      GRP_14       0.67      0.06      0.11        34
      GRP_15       0.00      0.00      0.00        10
      GRP_16       0.33      0.04      0.07        27
      GRP_17       0.94      0.94      0.94        17
      GRP_18       1.00      0.05      0.10        20
      GRP_19       0.50      0.06      0.11        65
       GRP_2       0.79      0.28      0.41        69
      GRP_20       0.00      0.00      0.00         6
      GRP_21       0.00      0.00      0.00         8
      GRP_22       0.00      0.00      0.00        10
      GRP_23       0.00      0.00      0.00         9
      GRP_24       0.70    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# We sorted the basis F1 score. Now we have four super groups
# Modelling excersice was done on each group so we have four models now
Group1- ['GRP_17','GRP_0','GRP_24','GRP_8']
Group2- ['GRP_2','GRP_5','GRP_12','GRP_13','GRP_16']
Group3- ['GRP_7','GRP_33','GRP_6','GRP_25']
Group4- ['GRP_19','GRP_3','GRP_9','GRP_10']

# Dividing the data into group 1

In [ ]:
from sklearn import metrics

In [ ]:
check_x = check.copy()

In [ ]:
#check_x.shape

(8164, 11)

In [ ]:
grp_list = ['GRP_17','GRP_0','GRP_24','GRP_8']

In [ ]:
check_x.rename(columns= {'Assignment group':'Assig_group'} , inplace =True)

In [ ]:

remove_list =['GRP_73','GRP_35','GRP_72','GRP_70','GRP_64','GRP_67','GRP_61','GRP_32','GRP_71','GRP_57','GRP_54','GRP_69']
remove_filt = (check_x['Assig_group'].isin(remove_list))
check_x= check_x.loc[~remove_filt]

In [ ]:
check_x['Assig_group'] = check_x['Assig_group'].apply(lambda x: x if x in grp_list else 'oth')

In [ ]:
check_x['Assig_group'].value_counts(dropna = False)

GRP_0     3785
oth       3351
GRP_8      661
GRP_24     289
GRP_17      78
Name: Assig_group, dtype: int64

In [ ]:
check_x.reset_index(drop=True,inplace=True)

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

corpus_x = []
for i in range(0, len(check_x)):
    review = check_x['p_long_desc'][i]
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_x.append(review)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v1=TfidfVectorizer(ngram_range=(1,3),min_df=2)
X=tfidf_v1.fit_transform(corpus_x).toarray()

In [ ]:
# Label encoder 
y=check_x['Assig_group']
y.value_counts(dropna=False)

GRP_0     3785
oth       3351
GRP_8      661
GRP_24     289
GRP_17      78
Name: Assig_group, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

GRP_0     3785
oth       3351
GRP_8      661
GRP_24     289
GRP_17      78
Name: Assig_group, dtype: int64

# Logistic Regression

In [ ]:
%%time
import gc 
from IPython.display import display, HTML, Markdown, clear_output
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ml_score_board = pd.DataFrame(columns=['Model'])
ml_model_dict = {}

#ml_pipelines = [Pipeline([('estimator',LogisticRegression(solver='lbfgs',random_state=0, multi_class='ovr'))])]
ml_pipelines = [Pipeline([('estimator',LogisticRegression(multi_class='multinomial', solver='newton-cg',random_state=0))])]
#ml_pipelines = [Pipeline([('estimator',OneVsOneClassifier(ExtraTreesClassifier()))])]
for ml_model_group1 in ml_pipelines:
  #estimator_name = ml_model['estimator'].__class__.__name__
  estimator_name = ml_model['estimator'].__class__.__name__

  if estimator_name == 'SVC':
    estimator_name = 'SVC-' + ml_model['estimator'].kernel
  print(f"\nTraining {estimator_name}...")

  ml_model_group1.fit(X_train,y_train)
  #ml_model_group1.fit(X_train_res,y_train_res)
  pred_build = ml_model_group1.predict(X_train)
  #pred_build = ml_model_group1.predict(X_train_res)
  score_build = metrics.accuracy_score(y_train, pred_build)
  #score_build = metrics.accuracy_score(y_train_res, pred_build)
  train_M = metrics.confusion_matrix(y_train, pred_build)
  #train_M = metrics.confusion_matrix(y_train_res, pred_build)
  pred = ml_model_group1.predict(X_test)
  score = metrics.accuracy_score(y_test, pred)
  test_M = metrics.confusion_matrix(y_test, pred,labels=['GRP_17','GRP_0','GRP_24','GRP_8'])
  clear_output(wait=True)
  ml_score_board = ml_score_board.append({'Model' : estimator_name, 'Build Acc' : score_build, 'Val Acc' : score},ignore_index=True)
  ml_model_dict[estimator_name] = ml_model
  gc.collect()
  display(ml_score_board)
  #print(train_M)
  #print(test_M)

,Model,Build Acc,Val Acc
0,LogisticRegression,0.864368,0.772245


CPU times: user 2min 1s, sys: 11.2 s, total: 2min 12s
Wall time: 33.7 s


In [ ]:
print(test_M)

[[ 16   1   0   0]
 [  1 991   0   0]
 [  0   5  43   0]
 [  0   2   1  63]]


In [ ]:
import pickle

In [ ]:
pickle.dump(ml_model_group1,open('/content/drive/MyDrive/Capstone_1/ml_model_group1.pkl','wb'))
pickle.dump(tfidf_v1,open('/content/drive/MyDrive/Capstone_1/tfidf_v1.pkl','wb'))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

       GRP_0       0.83      0.86      0.84      1159
      GRP_17       0.94      0.94      0.94        17
      GRP_24       0.90      0.50      0.64        86
       GRP_8       0.81      0.31      0.45       201
         oth       0.70      0.79      0.74       987

    accuracy                           0.77      2450
   macro avg       0.84      0.68      0.72      2450
weighted avg       0.78      0.77      0.76      2450



# ET

In [ ]:
%%time
import gc 
from IPython.display import display, HTML, Markdown, clear_output
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ml_score_board = pd.DataFrame(columns=['Model'])
ml_model_dict = {}

#ml_pipelines = [Pipeline([('estimator',LogisticRegression(solver='lbfgs',random_state=0, multi_class='ovr'))])]
#ml_pipelines = [Pipeline([('estimator',LogisticRegression(multi_class='multinomial', solver='newton-cg',random_state=0))])]
ml_pipelines = [Pipeline([('estimator',OneVsOneClassifier(ExtraTreesClassifier()))])]
for ml_model in ml_pipelines:
  #estimator_name = ml_model['estimator'].__class__.__name__
  estimator_name = ml_model['estimator'].__class__.__name__

  if estimator_name == 'SVC':
    estimator_name = 'SVC-' + ml_model['estimator'].kernel
  print(f"\nTraining {estimator_name}...")

  ml_model.fit(X_train,y_train)
  #ml_model.fit(X_train_res,y_train_res)
  pred_build = ml_model.predict(X_train)
  #pred_build = ml_model.predict(X_train_res)
  score_build = metrics.accuracy_score(y_train, pred_build)
  #score_build = metrics.accuracy_score(y_train_res, pred_build)
  train_M = metrics.confusion_matrix(y_train, pred_build)
  #train_M = metrics.confusion_matrix(y_train_res, pred_build)
  pred = ml_model.predict(X_test)
  score = metrics.accuracy_score(y_test, pred)
  test_M = metrics.confusion_matrix(y_test, pred)
  clear_output(wait=True)
  ml_score_board = ml_score_board.append({'Model' : estimator_name, 'Build Acc' : score_build, 'Val Acc' : score},ignore_index=True)
  ml_model_dict[estimator_name] = ml_model
  gc.collect()
  display(ml_score_board)
  #print(train_M)
  #print(test_M)

,Model,Build Acc,Val Acc
0,OneVsOneClassifier,0.944697,0.771429


CPU times: user 5min 43s, sys: 338 ms, total: 5min 43s
Wall time: 5min 42s


#It is here that we decide that we will continue with LR since LR has slightly better accuracy than ET

# Group1 model has accuracy of 77%

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

       GRP_0       0.81      0.87      0.84      1159
      GRP_17       0.94      1.00      0.97        17
      GRP_24       0.87      0.71      0.78        86
       GRP_8       0.81      0.38      0.52       201
         oth       0.71      0.74      0.73       987

    accuracy                           0.77      2450
   macro avg       0.83      0.74      0.77      2450
weighted avg       0.77      0.77      0.77      2450



# Dividing the data into group 2 xxxx

In [ ]:
check_x = check.copy()
check_x.rename(columns= {'Assignment group':'Assig_group'} , inplace =True)
remove_list =['GRP_73','GRP_35','GRP_72','GRP_70','GRP_64','GRP_67','GRP_61','GRP_32','GRP_71','GRP_57','GRP_54','GRP_69',
              'GRP_17','GRP_0','GRP_24','GRP_8']
remove_filt = (check_x['Assig_group'].isin(remove_list))
check_x= check_x.loc[~remove_filt]

In [ ]:
grp_list_second = ['GRP_2','GRP_5','GRP_12','GRP_13','GRP_16']
check_x['Assig_group'] = check_x['Assig_group'].apply(lambda x: x if x in grp_list_second else 'oth')

In [ ]:
check_x['Assig_group'].value_counts(dropna = False)

oth       2533
GRP_2      238
GRP_12     232
GRP_13     140
GRP_5      128
GRP_16      80
Name: Assig_group, dtype: int64

In [ ]:
check_x.reset_index(drop=True,inplace=True)

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

corpus_x = []
for i in range(0, len(check_x)):
    review = check_x['p_long_desc'][i]
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_x.append(review)
   
   
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v2=TfidfVectorizer(ngram_range=(1,3),min_df=2)
X=tfidf_v2.fit_transform(corpus_x).toarray()

# Label encoder 
y=check_x['Assig_group']
y.value_counts(dropna=False)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


oth       2533
GRP_2      238
GRP_12     232
GRP_13     140
GRP_5      128
GRP_16      80
Name: Assig_group, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

# Trying Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler
os =  RandomOverSampler(sampling_strategy='not majority')
X_train_res, y_train_res = os.fit_sample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
%%time
import gc 
from IPython.display import display, HTML, Markdown, clear_output
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ml_score_board = pd.DataFrame(columns=['Model'])
ml_model_dict = {}

#ml_pipelines = [Pipeline([('estimator',LogisticRegression(solver='lbfgs',random_state=0, multi_class='ovr'))])]
ml_pipelines = [Pipeline([('estimator',LogisticRegression(multi_class='multinomial', solver='newton-cg',random_state=0))])]
#ml_pipelines = [Pipeline([('estimator',OneVsOneClassifier(ExtraTreesClassifier()))])]
for ml_model_group2 in ml_pipelines:
  #estimator_name = ml_model['estimator'].__class__.__name__
  estimator_name = ml_model_group2['estimator'].__class__.__name__

  if estimator_name == 'SVC':
    estimator_name = 'SVC-' + ml_model['estimator'].kernel
  print(f"\nTraining {estimator_name}...")

  #ml_model_group2.fit(X_train,y_train)
  ml_model_group2.fit(X_train_res,y_train_res)
  #pred_build = ml_model_group2.predict(X_train)
  pred_build = ml_model_group2.predict(X_train_res)
  #score_build = metrics.accuracy_score(y_train, pred_build)
  score_build = metrics.accuracy_score(y_train_res, pred_build)
  #train_M = metrics.confusion_matrix(y_train, pred_build)
  train_M = metrics.confusion_matrix(y_train_res, pred_build)
  pred = ml_model_group2.predict(X_test)
  score = metrics.accuracy_score(y_test, pred)
  test_M = metrics.confusion_matrix(y_test, pred, labels = ['GRP_2','GRP_5','GRP_12','GRP_13','GRP_16'])
  clear_output(wait=True)
  ml_score_board = ml_score_board.append({'Model' : estimator_name, 'Build Acc' : score_build, 'Val Acc' : score},ignore_index=True)
  ml_model_dict[estimator_name] = ml_model
  gc.collect()
  display(ml_score_board)
  #print(train_M)
  #print(test_M)

,Model,Build Acc,Val Acc
0,LogisticRegression,0.955044,0.753479


CPU times: user 3min 53s, sys: 9.94 s, total: 4min 3s
Wall time: 1min 1s


In [ ]:
print(test_M)

[[41  0  4  1  0]
 [ 1 29  0  0  0]
 [ 2  0 40  0  0]
 [ 0  2  0 28  0]
 [ 1  0  0  0 20]]


In [ ]:
pickle.dump(ml_model_group2,open('/content/drive/MyDrive/Capstone_1/ml_model_group2.pkl','wb'))
pickle.dump(tfidf_v2,open('/content/drive/MyDrive/Capstone_1/tfidf_v2.pkl','wb'))


# Group2 model has accuracy of 75%

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

      GRP_12       0.59      0.74      0.66        54
      GRP_13       0.56      0.60      0.58        47
      GRP_16       0.91      0.77      0.83        26
       GRP_2       0.58      0.61      0.59        67
       GRP_5       0.22      0.72      0.34        40
         oth       0.90      0.78      0.84       772

    accuracy                           0.75      1006
   macro avg       0.63      0.70      0.64      1006
weighted avg       0.82      0.75      0.78      1006



# Dividing the data into group 3

In [ ]:
check_x = check.copy()
check_x.rename(columns= {'Assignment group':'Assig_group'} , inplace =True)
remove_list =['GRP_73','GRP_35','GRP_72','GRP_70','GRP_64','GRP_67','GRP_61','GRP_32','GRP_71','GRP_57','GRP_54','GRP_69',
              'GRP_17','GRP_0','GRP_24','GRP_8',
              'GRP_2','GRP_5','GRP_12','GRP_13','GRP_16']
remove_filt = (check_x['Assig_group'].isin(remove_list))
check_x= check_x.loc[~remove_filt]
check_x.reset_index(drop=True,inplace=True)

In [ ]:
grp_list_third = ['GRP_7','GRP_33','GRP_6','GRP_25']
check_x['Assig_group'] = check_x['Assig_group'].apply(lambda x: x if x in grp_list_third else 'oth')

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

corpus_x = []
for i in range(0, len(check_x)):
    review = check_x['p_long_desc'][i]
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_x.append(review)
   
   
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v3=TfidfVectorizer(ngram_range=(1,3),min_df=2)
X=tfidf_v3.fit_transform(corpus_x).toarray()

# Label encoder 
y=check_x['Assig_group']
y.value_counts(dropna=False)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


oth       2059
GRP_6      183
GRP_25     116
GRP_33     107
GRP_7       68
Name: Assig_group, dtype: int64

In [ ]:
check_x['Assig_group'].value_counts(dropna = False)

oth       2059
GRP_6      183
GRP_25     116
GRP_33     107
GRP_7       68
Name: Assig_group, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
os =  RandomOverSampler(sampling_strategy='not majority')
X_train_res, y_train_res = os.fit_sample(X_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
%%time
import gc 
from IPython.display import display, HTML, Markdown, clear_output
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ml_score_board = pd.DataFrame(columns=['Model'])
ml_model_dict = {}

#ml_pipelines = [Pipeline([('estimator',LogisticRegression(solver='lbfgs',random_state=0, multi_class='ovr'))])]
ml_pipelines = [Pipeline([('estimator',LogisticRegression(multi_class='multinomial', solver='newton-cg',random_state=0))])]
#ml_pipelines = [Pipeline([('estimator',OneVsOneClassifier(ExtraTreesClassifier()))])]
for ml_model_group3 in ml_pipelines:
  #estimator_name = ml_model['estimator'].__class__.__name__
  estimator_name = ml_model_group3['estimator'].__class__.__name__

  if estimator_name == 'SVC':
    estimator_name = 'SVC-' + ml_model_group3['estimator'].kernel
  print(f"\nTraining {estimator_name}...")

  #ml_model_group3.fit(X_train,y_train)
  ml_model_group3.fit(X_train_res,y_train_res)
  #pred_build = ml_model_group3.predict(X_train)
  pred_build = ml_model_group3.predict(X_train_res)
  #score_build = metrics.accuracy_score(y_train, pred_build)
  score_build = metrics.accuracy_score(y_train_res, pred_build)
  #train_M = metrics.confusion_matrix(y_train, pred_build)
  train_M = metrics.confusion_matrix(y_train_res, pred_build)
  pred = ml_model_group3.predict(X_test)
  score = metrics.accuracy_score(y_test, pred)
  test_M = metrics.confusion_matrix(y_test, pred,labels=['GRP_7','GRP_33','GRP_6','GRP_25'])
  clear_output(wait=True)
  ml_score_board = ml_score_board.append({'Model' : estimator_name, 'Build Acc' : score_build, 'Val Acc' : score},ignore_index=True)
  ml_model_dict[estimator_name] = ml_model
  gc.collect()
  display(ml_score_board)
  #print(train_M)
  #print(test_M)

,Model,Build Acc,Val Acc
0,LogisticRegression,0.960776,0.821053


CPU times: user 1min 11s, sys: 7.41 s, total: 1min 19s
Wall time: 20.2 s


In [ ]:
print(test_M)

[[17  0  0  0]
 [ 0 17  0  1]
 [ 0  0 41  0]
 [ 0  2  0 30]]


In [ ]:
pickle.dump(ml_model_group3,open('/content/drive/MyDrive/Capstone_1/ml_model_group3.pkl','wb'))
pickle.dump(tfidf_v3,open('/content/drive/MyDrive/Capstone_1/tfidf_v3.pkl','wb'))

# Group3 model has accuracy of 82%

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

      GRP_25       0.62      0.77      0.69        39
      GRP_33       0.59      0.53      0.56        32
       GRP_6       0.39      0.77      0.52        53
       GRP_7       0.77      0.85      0.81        20
         oth       0.94      0.84      0.89       616

    accuracy                           0.82       760
   macro avg       0.66      0.75      0.69       760
weighted avg       0.86      0.82      0.83       760



# Dividing the data into group 4

In [ ]:
check_x = check.copy()
check_x.rename(columns= {'Assignment group':'Assig_group'} , inplace =True)
remove_list =['GRP_73','GRP_35','GRP_72','GRP_70','GRP_64','GRP_67','GRP_61','GRP_32','GRP_71','GRP_57','GRP_54','GRP_69',
              'GRP_17','GRP_0','GRP_24','GRP_8',
              'GRP_2','GRP_5','GRP_12','GRP_13','GRP_16',
              'GRP_7','GRP_33','GRP_6','GRP_25']
remove_filt = (check_x['Assig_group'].isin(remove_list))
check_x= check_x.loc[~remove_filt]
check_x.reset_index(drop=True,inplace=True)

In [ ]:
grp_list_fourth = ['GRP_19','GRP_3','GRP_9','GRP_10']
check_x['Assig_group'] = check_x['Assig_group'].apply(lambda x: x if x in grp_list_fourth else 'oth')

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

corpus_x = []
for i in range(0, len(check_x)):
    review = check_x['p_long_desc'][i]
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_x.append(review)
   
   
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v4=TfidfVectorizer(ngram_range=(1,3),min_df=2)
X=tfidf_v4.fit_transform(corpus_x).toarray()

# Label encoder 
y=check_x['Assig_group']
y.value_counts(dropna=False)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


oth       1263
GRP_9      251
GRP_19     211
GRP_3      200
GRP_10     134
Name: Assig_group, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
os =  RandomOverSampler(sampling_strategy='not majority')
X_train_res, y_train_res = os.fit_sample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
%%time
import gc 
from IPython.display import display, HTML, Markdown, clear_output
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ml_score_board = pd.DataFrame(columns=['Model'])
ml_model_dict = {}

#ml_pipelines = [Pipeline([('estimator',LogisticRegression(solver='lbfgs',random_state=0, multi_class='ovr'))])]
ml_pipelines = [Pipeline([('estimator',LogisticRegression(multi_class='multinomial', solver='newton-cg',random_state=0))])]
#ml_pipelines = [Pipeline([('estimator',OneVsOneClassifier(ExtraTreesClassifier()))])]
for ml_model_group4 in ml_pipelines:
  #estimator_name = ml_model['estimator'].__class__.__name__
  estimator_name = ml_model_group4['estimator'].__class__.__name__

  if estimator_name == 'SVC':
    estimator_name = 'SVC-' + ml_model['estimator'].kernel
  print(f"\nTraining {estimator_name}...")

  #ml_model_group4.fit(X_train,y_train)
  ml_model_group4.fit(X_train_res,y_train_res)
  #pred_build = ml_model.predict(X_train)
  pred_build = ml_model_group4.predict(X_train_res)
  #score_build = metrics.accuracy_score(y_train, pred_build)
  score_build = metrics.accuracy_score(y_train_res, pred_build)
  #train_M = metrics.confusion_matrix(y_train, pred_build)
  train_M = metrics.confusion_matrix(y_train_res, pred_build)
  pred = ml_model_group4.predict(X_test)
  score = metrics.accuracy_score(y_test, pred)
  test_M = metrics.confusion_matrix(y_test, pred,labels=['GRP_19','GRP_3','GRP_9','GRP_10'])
  clear_output(wait=True)
  ml_score_board = ml_score_board.append({'Model' : estimator_name, 'Build Acc' : score_build, 'Val Acc' : score},ignore_index=True)
  ml_model_dict[estimator_name] = ml_model
  gc.collect()
  display(ml_score_board)
  #print(train_M)
  #print(test_M)

,Model,Build Acc,Val Acc
0,LogisticRegression,0.882579,0.731392


CPU times: user 40.4 s, sys: 4.48 s, total: 44.8 s
Wall time: 11.5 s


In [ ]:
print(test_M)

[[24 12  0  0]
 [10 27  0  0]
 [ 1  1 61  0]
 [ 0  1 18  8]]


In [ ]:
pickle.dump(ml_model_group4,open('/content/drive/MyDrive/Capstone_1/ml_model_group4.pkl','wb'))
pickle.dump(tfidf_v4,open('/content/drive/MyDrive/Capstone_1/tfidf_v4.pkl','wb'))

# Group4 model has accuracy of 73%

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

      GRP_10       0.80      0.18      0.30        44
      GRP_19       0.53      0.39      0.45        61
       GRP_3       0.43      0.47      0.45        58
       GRP_9       0.66      0.80      0.73        76
         oth       0.81      0.88      0.84       379

    accuracy                           0.73       618
   macro avg       0.65      0.54      0.55       618
weighted avg       0.73      0.73      0.71       618



# Others

In [ ]:
check_x = check.copy()
check_x.rename(columns= {'Assignment group':'Assig_group'} , inplace =True)
remove_list =['GRP_73','GRP_35','GRP_72','GRP_70','GRP_64','GRP_67','GRP_61','GRP_32','GRP_71','GRP_57','GRP_54','GRP_69',
              'GRP_17','GRP_0','GRP_24','GRP_8',
              'GRP_2','GRP_5','GRP_12','GRP_13','GRP_16'
              'GRP_7','GRP_33','GRP_6','GRP_25',
              'GRP_19','GRP_3','GRP_9','GRP_10']
remove_filt = (check_x['Assig_group'].isin(remove_list))
check_x= check_x.loc[~remove_filt]
check_x.reset_index(drop=True,inplace=True)

In [ ]:
check_x['Assig_group'].value_counts(dropna= False)

GRP_14    115
GRP_4      99
GRP_29     94
GRP_18     87
GRP_16     80
GRP_7      68
GRP_31     62
GRP_34     61
GRP_26     54
GRP_40     45
GRP_28     44
GRP_41     40
GRP_15     38
GRP_42     37
GRP_20     35
GRP_45     34
GRP_22     31
GRP_1      31
GRP_11     29
GRP_21     28
GRP_47     27
GRP_62     25
GRP_23     23
GRP_60     20
GRP_39     19
GRP_27     18
GRP_30     18
GRP_37     16
GRP_36     15
GRP_44     15
GRP_50     13
GRP_53     11
GRP_65     11
GRP_51      8
GRP_52      8
GRP_55      8
GRP_48      7
GRP_43      5
GRP_46      5
GRP_59      5
GRP_66      4
GRP_58      3
GRP_49      3
GRP_38      3
GRP_63      3
GRP_56      3
GRP_68      3
Name: Assig_group, dtype: int64

In [ ]:

from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

corpus_x = []
for i in range(0, len(check_x)):
    review = check_x['p_long_desc'][i]
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_x.append(review)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
 from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_voth=TfidfVectorizer(ngram_range=(1,3),min_df=2)
X=tfidf_voth.fit_transform(corpus_x).toarray()

# Label encoder 
y=check_x['Assig_group']
y.value_counts(dropna=False)

GRP_14    115
GRP_4      99
GRP_29     94
GRP_18     87
GRP_16     80
GRP_7      68
GRP_31     62
GRP_34     61
GRP_26     54
GRP_40     45
GRP_28     44
GRP_41     40
GRP_15     38
GRP_42     37
GRP_20     35
GRP_45     34
GRP_22     31
GRP_1      31
GRP_11     29
GRP_21     28
GRP_47     27
GRP_62     25
GRP_23     23
GRP_60     20
GRP_39     19
GRP_27     18
GRP_30     18
GRP_37     16
GRP_36     15
GRP_44     15
GRP_50     13
GRP_53     11
GRP_65     11
GRP_51      8
GRP_52      8
GRP_55      8
GRP_48      7
GRP_43      5
GRP_46      5
GRP_59      5
GRP_66      4
GRP_58      3
GRP_49      3
GRP_38      3
GRP_63      3
GRP_56      3
GRP_68      3
Name: Assig_group, dtype: int64

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
os =  RandomOverSampler(sampling_strategy='not majority')
X_train_res, y_train_res = os.fit_sample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(y_train)))
print('Resampled dataset shape {}'.format(Counter(y_train_res)))

Original dataset shape Counter({'GRP_14': 75, 'GRP_29': 69, 'GRP_4': 65, 'GRP_16': 64, 'GRP_18': 60, 'GRP_31': 47, 'GRP_34': 44, 'GRP_26': 42, 'GRP_7': 41, 'GRP_28': 33, 'GRP_40': 32, 'GRP_41': 29, 'GRP_42': 26, 'GRP_15': 24, 'GRP_22': 23, 'GRP_45': 21, 'GRP_1': 21, 'GRP_21': 20, 'GRP_23': 19, 'GRP_47': 19, 'GRP_11': 18, 'GRP_20': 18, 'GRP_62': 18, 'GRP_39': 16, 'GRP_60': 14, 'GRP_30': 13, 'GRP_27': 13, 'GRP_36': 11, 'GRP_44': 11, 'GRP_53': 10, 'GRP_37': 9, 'GRP_50': 8, 'GRP_65': 8, 'GRP_51': 7, 'GRP_48': 6, 'GRP_55': 6, 'GRP_52': 5, 'GRP_46': 3, 'GRP_68': 3, 'GRP_56': 3, 'GRP_59': 3, 'GRP_58': 2, 'GRP_63': 2, 'GRP_43': 2, 'GRP_49': 2, 'GRP_66': 1, 'GRP_38': 1})
Resampled dataset shape Counter({'GRP_28': 75, 'GRP_36': 75, 'GRP_53': 75, 'GRP_29': 75, 'GRP_14': 75, 'GRP_11': 75, 'GRP_50': 75, 'GRP_40': 75, 'GRP_18': 75, 'GRP_16': 75, 'GRP_30': 75, 'GRP_23': 75, 'GRP_15': 75, 'GRP_7': 75, 'GRP_20': 75, 'GRP_21': 75, 'GRP_34': 75, 'GRP_42': 75, 'GRP_45': 75, 'GRP_4': 75, 'GRP_44': 75, 'GRP

In [ ]:

%%time
import gc 
from IPython.display import display, HTML, Markdown, clear_output
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ml_score_board = pd.DataFrame(columns=['Model'])
ml_model_dict = {}

#ml_pipelines = [Pipeline([('estimator',LogisticRegression(solver='lbfgs',random_state=0, multi_class='ovr'))])]
ml_pipelines = [Pipeline([('estimator',LogisticRegression(multi_class='multinomial', solver='newton-cg',random_state=0))])]
#ml_pipelines = [Pipeline([('estimator',OneVsOneClassifier(ExtraTreesClassifier()))])]
for ml_model_oth in ml_pipelines:
  #estimator_name = ml_model['estimator'].__class__.__name__
  estimator_name = ml_model['estimator'].__class__.__name__

  if estimator_name == 'SVC':
    estimator_name = 'SVC-' + ml_model['estimator'].kernel
  print(f"\nTraining {estimator_name}...")

  #ml_model.fit(X_train,y_train)
  ml_model_oth.fit(X_train_res,y_train_res)
  #pred_build = ml_model.predict(X_train)
  pred_build = ml_model_oth.predict(X_train_res)
  #score_build = metrics.accuracy_score(y_train, pred_build)
  score_build = metrics.accuracy_score(y_train_res, pred_build)
  #train_M = metrics.confusion_matrix(y_train, pred_build)
  train_M = metrics.confusion_matrix(y_train_res, pred_build)
  pred = ml_model_oth.predict(X_test)
  score = metrics.accuracy_score(y_test, pred)
  test_M = metrics.confusion_matrix(y_test, pred)
  clear_output(wait=True)
  ml_score_board = ml_score_board.append({'Model' : estimator_name, 'Build Acc' : score_build, 'Val Acc' : score},ignore_index=True)
  ml_model_dict[estimator_name] = ml_model_oth
  gc.collect()
  display(ml_score_board)
  #print(train_M)
  #print(test_M)
  
  

,Model,Build Acc,Val Acc
0,LogisticRegression,0.937021,0.497642


CPU times: user 51.2 s, sys: 10.4 s, total: 1min 1s
Wall time: 15.8 s


In [ ]:
pickle.dump(ml_model_oth,open('/content/drive/MyDrive/Capstone_1/ml_model_oth.pkl','wb'))
pickle.dump(tfidf_voth,open('/content/drive/MyDrive/Capstone_1/tfidf_voth.pkl','wb'))

# Model for rest of the group has an accuracy of 47%

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

       GRP_1       0.38      0.30      0.33        10
      GRP_11       0.38      0.45      0.42        11
      GRP_14       0.59      0.42      0.49        40
      GRP_15       0.20      0.14      0.17        14
      GRP_16       1.00      0.62      0.77        16
      GRP_18       0.57      0.59      0.58        27
      GRP_20       0.14      0.12      0.13        17
      GRP_21       0.50      0.25      0.33         8
      GRP_22       0.33      0.38      0.35         8
      GRP_23       1.00      1.00      1.00         4
      GRP_26       0.35      0.50      0.41        12
      GRP_27       0.14      0.20      0.17         5
      GRP_28       0.24      0.36      0.29        11
      GRP_29       0.57      0.64      0.60        25
      GRP_30       0.00      0.00      0.00         5
      GRP_31       0.46      0.40      0.43        15
      GRP_34       0.45      0.53      0.49        17
      GRP_36       0.80    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Final inference : 
Now we have 4 models whose average accuracy is around 77% and covers around 87% of the entire data.

A fifth model was built for remaining groups. The frequency of these groups was too low to build a model which gives high accuracy. However still our last model gave 50% of accuracy